<a href="https://colab.research.google.com/github/Gebrehiwot-Tesfaye/CRUD-MySQL-DB/blob/main/MSc_Thesis_coffee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/noamaanabdulazeem/jmuben-coffee-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gebrehiwottesfaye
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/noamaanabdulazeem/jmuben-coffee-dataset


100%|██████████| 773M/773M [00:09<00:00, 87.3MB/s]


In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/biniyamyoseph/ethiopian-coffee-leaf-disease")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gebrehiwottesfaye
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/biniyamyoseph/ethiopian-coffee-leaf-disease


100%|██████████| 1.91G/1.91G [00:55<00:00, 36.7MB/s]


In [3]:
pip install tensorflow tensorflow-federated opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 95.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce

In [1]:
!pip install -U jaxlib

In [1]:
!pip install --upgrade jax jaxlib

In [1]:
!pip install --upgrade tensorflow numpy

  Using cached numpy-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import opendatasets as od

# Download the dataset
od.download("https://www.kaggle.com/datasets/biniyamyoseph/ethiopian-coffee-leaf-disease")

# Define dataset path
dataset_path = "./ethiopian-coffee-leaf-disease"

# Preprocess data
IMG_SIZE = (128, 128)  # Resize images
BATCH_SIZE = 32

# Use ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode='categorical'
)


Skipping, found downloaded files in "./ethiopian-coffee-leaf-disease" (use force=True to force download)
Found 9600 images belonging to 1 classes.
Found 2400 images belonging to 1 classes.


In [1]:
!pip install tensorflow_federated

In [ ]:
import os
import tensorflow as tf
import tensorflow_federated as tff
import opendatasets as od

# Download the dataset (if not already downloaded)
od.download("https://www.kaggle.com/datasets/biniyamyoseph/ethiopian-coffee-leaf-disease",force=True)



# Print directory structure for debugging
dataset_path = "./ethiopian-coffee-leaf-disease/ethiopian_cofee_leaf_dataset"


test_path = os.path.join(dataset_path, 'test')
print("Directories in test_path:", os.listdir(test_path) if os.path.exists(test_path) else "Test path not found!")


IMG_SIZE = (128, 128)  # Resize images
BATCH_SIZE = 32
NUM_CLIENTS = 5  # Number of simulated clients
NUM_EPOCHS = 5  # Number of federated training rounds

# Function to preprocess a single image
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    return img, label

# Function to create a client dataset
def create_client_dataset(client_id, image_paths, labels):
    start_index = client_id * len(image_paths) // NUM_CLIENTS
    end_index = (client_id + 1) * len(image_paths) // NUM_CLIENTS
    client_image_paths = image_paths[start_index:end_index]
    client_labels = labels[start_index:end_index]

    # Create TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((client_image_paths, client_labels))
    dataset = dataset.map(preprocess_image).batch(BATCH_SIZE)
    return dataset



def load_dataset():
    # Dynamically identify the test folder
    possible_test_paths = [os.path.join(dataset_path, 'test')]
    for path in possible_test_paths:
        if os.path.exists(path) and os.path.isdir(path):
            test_path = path
            break
    else:
        raise FileNotFoundError(f"Test path not found in the provided dataset directory: {dataset_path}")

    # Load categories and images
    categories = os.listdir(test_path)
    image_paths = []
    labels = []
    for idx, category in enumerate(categories):
        category_dir = os.path.join(test_path, category)
        if os.path.isdir(category_dir):
            category_image_paths = [os.path.join(category_dir, fname)
                                    for fname in os.listdir(category_dir)
                                    if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
            image_paths.extend(category_image_paths)
            labels.extend([idx] * len(category_image_paths))

    return image_paths, labels


# Call the function
try:
    image_paths, labels = load_dataset()
    print(f"Loaded {len(image_paths)} images with labels {set(labels)}")
except FileNotFoundError as e:
    print(e)


NUM_CLIENTS = 5  # Define number of federated clients

image_paths, labels = load_dataset()

def create_client_dataset(client_id, image_paths, labels):
    # Create datasets for individual clients
    print(f"Creating dataset for client {client_id}")
    return {
        "client_id": client_id,
        "images": image_paths[client_id::NUM_CLIENTS],  # Example logic to divide data
        "labels": labels[client_id::NUM_CLIENTS],
        "image_paths": image_paths,

    }

# Create federated train data
federated_train_data = [
    create_client_dataset(client_id, image_paths, labels)
    for client_id in range(NUM_CLIENTS)
]


# Define the CNN model
def create_keras_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(set(labels)), activation='softmax')  # Multi-class classification
    ])
    return model

# Wrap the Keras model for TFF
def model_fn():
    keras_model = create_keras_model()

    # Use the `element_spec` of one client dataset
    input_spec = federated_train_data[0].element_spec

    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )


# Define the federated averaging algorithm
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
)

# Initialize the state
state = iterative_process.initialize()

# Perform federated training
for round_num in range(1, NUM_EPOCHS + 1):
    client_data = [client for client in federated_train_data]  # Simulate client data
    state, metrics = iterative_process.next(state, client_data)
    print(f'Round {round_num}, Metrics: {metrics}')

# Function to predict on a single image
def predict_image(image_path, model):
    img = preprocess_image(image_path, label=0)[0]  # Ignore label
    img_array = tf.expand_dims(img, axis=0)
    prediction = model.predict(img_array)
    predicted_class = tf.argmax(prediction, axis=1).numpy()  # Multi-class prediction
    return predicted_class[0]

# Get the trained Keras model from the TFF state
trained_keras_model = create_keras_model()
state.model.assign_weights_to(trained_keras_model)

# Example usage: Predict on an image from the test data
test_image_path = image_paths[0]  # Use the first image for testing
predicted_class = predict_image(test_image_path, trained_keras_model)
print(f"Predicted class: {predicted_class}")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: